In [1]:
import os
import os.path as op
import glob
import numpy as np
import multiprocessing
from nipype.interfaces import fsl

In [52]:
#preliminary housekeeping
home_dir = '/data/home/iballard/mag/'
subj_file = home_dir + 'subjects.txt'
subs = list(np.loadtxt(subj_file,'string'))
os.chdir(home_dir)
exps = ['mag']
runs = map(str,range(1,5))

In [62]:
print len(subs)

19


In [63]:
#get cortical areas from Yeo atlas
path = home_dir + '/data/fsaverage/label/'

#get cortical areas from Michael's parcellation
cortical_areas =  glob.glob(path + '*yeo*')
cortical_areas = [x.split('/')[-1] for x in cortical_areas]

#get vmpfc roi
# cortical_areas =  glob.glob(path + '*vmpfc_bartre.label')
# cortical_areas = [x.split('/')[-1] for x in cortical_areas]
# cortical_areas

In [58]:
def make_mask(in_tuple):
    sub,roi = in_tuple
    roi_id = roi.split('.')[1]
    hemi = roi.split('.')[0]
    
    cmd = ['make_masks.py',
           '-s',
           sub,
           '-roi',
           hemi + '.' + roi_id,
            '-label',
           roi_id,
           '-sample graymid',
           '-hemi',
           hemi,
           '-debug',
           '-serial']
    cmd = ' '.join(cmd)
    os.system(cmd)

In [59]:
in_tuples = []
for sub in subs:
    for roi in cortical_areas:
        in_tuples.append((sub,roi))

In [64]:
pool = multiprocessing.Pool(50)
pool.map(make_mask,in_tuples)
pool.terminate()
pool.join()

In [ ]:
##bash commands for warping MNI brain vmPFC roi to freesurfer space
# mri_vol2surf --mov vmpfc_bartre_5way.nii.gz --reg MNI_to_fsaverage.dat --projdist-max 0 1 .1 --interp nearest  --hemi lh --out lh.vmpfc_bartre.mgh --reshape --trgsubject fsaverage --cortex

# mri_vol2surf --mov vmpfc_bartre_5way.nii.gz --reg MNI_to_fsaverage.dat --projdist-max 0 1 .1 --interp nearest  --hemi rh --out rh.vmpfc_bartre.mgh --reshape --trgsubject fsaverage --cortex

# mri_binarize --i rh.vmpfc_bartre.mgh --min .1 --o rh.vmpfc_bin.mgh
# mri_binarize --i lh.vmpfc_bartre.mgh --min .1 --o lh.vmpfc_bin.mgh

# mri_cor2label --i lh.vmpfc_bin.mgh --l label/lh.vmpfc_bartre.label --surf fsaverage lh --id 1
# mri_cor2label --i rh.vmpfc_bin.mgh --l label/rh.vmpfc_bartre.label --surf fsaverage lh --id 1